In [83]:
import numpy as np
import pandas as pd
from datetime import datetime
import os
from tqdm.notebook import tqdm
import pickle
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns',100)

In [3]:
#Esta función devuelve una tupla con la ruta del fichero y luego nos quedamos con la 'fecha' del fichero
def file_iterator(path):
    """
    DESCRIPTION
      This function retreives all files in a specific folder and save the result in a pickle file
    ARGUMENTS
      path: Where we want to get all files
    RETURN
      A list of tuples, first element is the path and the second is the file name without any extension
    """
    
    return [(os.path.join(path,sub,file),file.split('.')[0]) for sub in os.listdir(path) 
            for file in os.listdir(os.path.join(path,sub))]

In [4]:
#Esta función crea un pickle para saber el último refresco del fichero de stock
def file_to_pickle(files, pkl_file):
    """
    DESCRIPTION
      This function saves the files tuple in a pickle. We would use it as 'latest stock files updated'. Then we
      just need to check the difference once we add a new stock file and pass those new ones to the update DataFrame
      function
    ARGUMENTS
      files: This is list of tuples, first element is the path and the second is the file name without any extension
      pkl_file: Pickle file name
    RETURN
      Nothing, it will create a pickle file
    """
    
    pkl = open(pkl_file, 'wb')
    pickle.dump(files, pkl)                     
    pkl.close()

In [5]:
#Esta función recupera el último estado del stock almanecado en un pickle
def pickle_to_file(pkl_file):
    """
    DESCRIPTION
      This function retrieves the pickle files tuple and return it
    ARGUMENTS
      pkl_file: Pickle file name
    RETURN
      A list of tuples, first element is the path and the second is the file name without any extension
    """    
    
    pkl = open(pkl_file, 'rb')     
    files = pickle.load(pkl)
    pkl.close()
    return files

In [6]:
#Comprueba que el fichero pickle existe
def pickle_exist(pkl_file):
    """
    DESCRIPTION
      This function check if the pickle file exists
    ARGUMENTS
      pkl_file: Pickle file name
    RETURN
      True or False
    """      
    
    if os.path.isfile(pkl_file):
        return True
    else:
        return False

In [64]:
#Devuelve el listado de los nuevos ficheros de stock que hay que añadir
def file_checker(path, pkl_file):
    """
    DESCRIPTION
      This function check if there is new stock files and return the new list of new files
      Good learning from here: https://www.geeksforgeeks.org/python-intersection-two-lists/
    ARGUMENTS
      path: Where we want to get all new files
      pkl_file: Pickle file name
    RETURN
      A list of tuples, first element is the path and the second is the file name without any extension
    """
    
    #https://www.geeksforgeeks.org/python-intersection-two-lists/
    #Create two sets one with the lates pickle file and the current file and get the difference
    current_files = pickle_to_file(pkl_file)
    new_files = file_iterator(path)
    #Return the difference of files
    return list(set(new_files) - set(current_files))

In [54]:
#Devuelve un DataFrame con toda la información que necesitamos
def file_extractor(files):
    """
    DESCRIPTION
      This function return a concatenated DataFrame with all desired columns
    ARGUMENTS
      files: This is list of tuples, first element is the path and the second is the file name without any extension
    RETURN
      A concatenated DataFrame with 10 columns based on the files contained in the argument
    """
    
    #A list of DataFrames, where then we will concatenated
    stock = []
    #This is the returned DataFrame
    pd_stock = pd.DataFrame()
    
    #Progression bar
    pbar = tqdm()
    pbar.reset(len(files))
    
    #Files is a Tuple made up of (path,name without extension)
    for path, name in files:
        #Read the file
        aux_stock = pd.read_excel(path, sheet_name='Daily Stock Report')        
        
        #Get the desired columns and this is difficult because the format has changed across years
        #Format 2 has this condition aux_stock.columns[0] == 'Country' else Format 1
        if aux_stock.columns[0] != 'Country':
            #First of all pick the Tablets from the old version - Condition NaN Column 2 and Tablet Column 5
            aux_stock_tablet = aux_stock[(aux_stock.iloc[:,4].str.lower() == 'tablet') 
                                         & (aux_stock.iloc[:,1].isna())].iloc[:,[1,2,3,4,5,9]]
            #Need to add Spain in one column
            aux_stock_tablet.fillna('Spain', inplace=True)

            #Secondly, get the rest of the products from Spain
            aux_stock = aux_stock[aux_stock.iloc[:,1] == 'Spain'].iloc[:,[1,2,3,4,5,9]]
            #Merge both DataFrame
            aux_stock = pd.concat([aux_stock_tablet, aux_stock], ignore_index=True)
        else:
            #There are a lot of conditions which has changed across years
            aux_stock = aux_stock[(aux_stock.iloc[:,0] == 'Shared') | (aux_stock.iloc[:,0] == 'Shared - Group II') 
                                  | (aux_stock.iloc[:,0].str.contains('ES')) 
                                  | (aux_stock.iloc[:,0] == 'Spain')].iloc[:,[0,1,2,3,4,8]]
        
            
        #We will use the file name for creating the Date, Year, Month, Day and Week Number
        year, month, day = name.split('-')
        #Important we need to keep the ISO_Year, so the year from the file sometimes is not correct
        year, week, weekday = pd.to_datetime(name).isocalendar()
        #Add the Date columns
        aux_stock['Year'] = int(year)
        aux_stock['Month'] = int(month)
        aux_stock['Day'] = int(day)
        aux_stock['Week'] = week
        if week < 10:
            aux_stock['Year-Week'] = str(year) + '-0' + str(week)
        else:
            aux_stock['Year-Week'] = str(year) + '-' + str(week)
        
        #We add this NUMPY ARRAY because the columns have different names
        stock.append(aux_stock.values)
        #Progress bar
        pbar.update()
    
    #Progress bar
    pbar.refresh()
    #Concatenated all NUMPY arrays and convert to DataFrame  
    pd_stock = pd.DataFrame(np.concatenate(stock))
    
    #New the columns name because they are unknow, just 0, 1, 2...
    columns = ['Country', 'PN', 'Product', 
               'Brand', 'Stock', 'Available', 'Year', 'Month', 'Day', 'Week', 'Year-Week']
    pd_stock.columns = columns
    #Return the final DataFrame in the desired order
    return pd_stock[['Year', 'Month', 'Day', 'Week','Year-Week', 'Country', 'PN', 'Product', 
               'Brand', 'Stock', 'Available']].sort_values(by=['Year', 'Month', 'Day'])

In [9]:
#Comprueba que el fichero Excel existe
def excel_exist(excel_file):
    """
    DESCRIPTION
      This function check if the stock Excel file exists
    ARGUMENTS
      excel_file: Stock Excel file name
    RETURN
      True or False
    """      
    
    if os.path.isfile(excel_file):
        return True
    else:
        return False

In [10]:
#Convierte el DataFrame en un fichero Excel o lo añade al último creado
def file_to_excel(stock, excel_file):
    """
    DESCRIPTION
      This function creates the stock Excel file. If could be just the full stock file or an append
    ARGUMENTS
      stock: This is the latest stock DataFrame to convert to Excel or to append to the current stock file
      excel_file: Stock Excel file name
    RETURN
      Void
    """       

    if excel_exist(excel_file):
        stock = pd.concat([stock, pd.read_excel(excel_file)], ignore_index=True)
        stock.sort_values(by=['Year', 'Month', 'Day'], inplace=True)
    stock.to_excel(excel_file, index=False)

In [11]:
#Devuelve un DataFrame de un fichero excel
def excel_to_DataFrame(excel_file):
    """
    DESCRIPTION
      This function read an Excel file and return a DataFrame
    ARGUMENTS
      excel_file: Stock Excel file name
    RETURN
      DataFrame with the stock
    """ 
    
    return pd.read_excel(excel_file)

In [12]:
#Esto deja el fichero de stock niquelado
def file_cleanup(stock):
    """
    DESCRIPTION
      This function cleanup the stock file because it has TABLET instead of Tablets and there are some NaN in Brand
      which have to rename for Phone and Option abd some NaN in Description... which has to be Tablet
    ARGUMENTS
      stock: This is the Mega DataFrame with all stock files aggreggated 
    RETURN
      File cleanup with the new scheme
    """
    
    stock.loc[(stock['Brand'].isnull()) & (stock['PN'].str.contains('PA')), 'Brand'] = 'Phone'
    stock.loc[(stock['Brand'].isnull()) & (stock['PN'].str.contains('GX')), 'Brand'] = 'Option'
    stock.loc[(stock['Product'].isnull()) & (stock['PN'].str.contains('ZA')), 'Product'] = 'Tablet'
    stock.loc[stock['Brand'] == 'TABLET', 'Brand'] = 'Tablet'
    #Return the cleaned stock file
    return stock

In [57]:
#Se queda con los duplicados de la última semana
def keep_latest_week(stock):
    """
    DESCRIPTION
      This function return the stock file with the latest week data
    ARGUMENTS
      stock: This is the Mega DataFrame with all stock files aggreggated 
    RETURN
      File with the latest week data
    """
    
    #Just two columns same week and same PartNumber/SKU as filter
    return stock[~stock.duplicated(keep = 'last', subset = ['Year-Week', 'PN'])].sort_values(by=['Year-Week'])

In [58]:
#Esta es la llamada principal que hace todo
def create_stock_file(path, pkl_file, excel_file):
    """
    DESCRIPTION
      This function create the stock Excel file from the scratch or add to the end
    ARGUMENTS
      path: Where we want to get all new files
      pkl_file: Pickle file name
      excel_file: Stock Excel file name
    RETURN
      Void
    """    
    
    #Get the new tuples of files based on pickle if empty, it creates the full list
    if not pickle_exist(pkl_file):
        files = file_iterator(path)
    else:
        files = file_checker(path, pkl_file)
    
    #If the List of Tuples is not empty, if there is files to add
    if files:
        #Create the DataFrame and create the Excel File
        stock = file_extractor(files)
        stock = file_cleanup(stock)
        stock = keep_latest_week(stock)
        file_to_excel(stock, excel_file)
    
    #Finally, we create the pickle file which is the latest stock file status
    files = file_iterator(path)
    file_to_pickle(files, pkl_file)

In [15]:
#Constants
stock_path = '/home/dsc/Repos/TFM/Stock'
stock_excel = 'stock.xlsx'
pickle_path = 'stock_files.pkl'

In [60]:
#This is the starting point. Everytime executes this program to update the stock file
create_stock_file(stock_path, pickle_path, stock_excel)

### DESDE AQUI

In [61]:
stock = excel_to_DataFrame(stock_excel)

In [86]:
stock[stock['Brand'] == 'Notebook'].groupby(['Year-Week'])['Available'].sum()

ValueError: key of type tuple not found and not a MultiIndex

In [69]:
aux = stock[stock['Brand'] == 'Notebook'].groupby(['Product', 'Year-Week'])['Available'].sum().unstack()

In [70]:
aux

Year-Week,2015-11,2015-12,2015-13,2015-14,2015-15,2015-16,2015-17,2015-18,2015-19,2015-20,2015-21,2015-22,2015-23,2015-24,2015-25,2015-26,2015-27,2015-28,2015-29,2015-30,2015-31,2015-32,2015-33,2015-34,2015-35,2015-36,2015-37,2015-38,2015-39,2015-40,2015-41,2015-42,2015-43,2015-44,2015-45,2015-46,2015-47,2015-48,2015-49,2015-50,2015-51,2015-52,2015-53,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,...,2020-19,2020-20,2020-21,2020-22,2020-23,2020-24,2020-25,2020-26,2020-27,2020-28,2020-29,2020-30,2020-31,2020-32,2020-33,2020-34,2020-35,2020-36,2020-37,2020-38,2020-39,2020-40,2020-41,2020-42,2020-43,2020-44,2020-45,2020-46,2020-47,2020-48,2020-49,2020-50,2020-51,2020-52,2020-53,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2021-13,2021-14,2021-15
Product,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100S-11IBY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,20.0,40.0,40.0,40.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110S-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320S-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ideapad 520S-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ideapad 720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [71]:
aux.fillna(value = 0, inplace = True)

In [78]:
aux['2015-11'].sum()


78.0

In [ ]:
aux.groupby(['Year','Month','Day'])[['Visits','Revenue']].sum()

In [31]:
pd.DataFrame(aux.columns)

,Year-Week
0,2015-11
1,2015-12
2,2015-13
3,2015-14
4,2015-15
...,...
309,2021-5
310,2021-6
311,2021-7
312,2021-8


In [43]:
type(aux.columns)

pandas.core.indexes.base.Index

In [49]:
aux[aux['2021-1'] == aux['2021-1']]

Year-Week,2015-11,2015-12,2015-13,2015-14,2015-15,2015-16,2015-17,2015-18,2015-19,2015-20,2015-21,2015-22,2015-23,2015-24,2015-25,2015-26,2015-27,2015-28,2015-29,2015-30,2015-31,2015-32,2015-33,2015-34,2015-35,2015-36,2015-37,2015-38,2015-39,2015-40,2015-41,2015-42,2015-43,2015-44,2015-45,2015-46,2015-47,2015-48,2015-49,2015-50,2015-51,2015-52,2015-53,2016-1,2016-10,2016-11,2016-12,2016-13,2016-14,2016-15,...,2020-22,2020-23,2020-24,2020-25,2020-26,2020-27,2020-28,2020-29,2020-3,2020-30,2020-31,2020-32,2020-33,2020-34,2020-35,2020-36,2020-37,2020-38,2020-39,2020-4,2020-40,2020-41,2020-42,2020-43,2020-44,2020-45,2020-46,2020-47,2020-48,2020-49,2020-5,2020-50,2020-51,2020-52,2020-53,2020-6,2020-7,2020-8,2020-9,2021-1,2021-10,2021-11,2021-2,2021-3,2021-4,2021-5,2021-6,2021-7,2021-8,2021-9
Product,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NB IP 3 14ARE05 R3 4G 4G 256G 10H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,10.0,10.0,10.0,8.0,7.0,6.0,NaN,6.0,3.0,1.0,1.0,0.0,0.0,148.0,44.0,6.0,3.0,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,-3.0,-15.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NB IP 3 14ARE05 R5 4G 4G 512G 10H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,20.0,20.0,20.0,20.0,20.0,20.0,NaN,20.0,17.0,17.0,117.0,115.0,113.0,71.0,28.0,4.0,4.0,NaN,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
NB IP 3 14IIL05 I3 4G 4G 256G 10H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,190.0,173.0,NaN,-1.0,-1.0,2.0,230.0,230.0,230.0,230.0,230.0,227.0,154.0,NaN,18.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NB IP 3 14IIL05 I5 4G 4G 512G 10H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,58.0,29.0,NaN,0.0,0.0,140.0,139.0,130.0,110.0,83.0,57.0,57.0,25.0,NaN,9.0,8.0,2.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
NB IP 3 14IIL05 I7 4G 4G 1T 10H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,54.0,50.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NoteBook TP T14s G1 I7 16G 10P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,5.0,3.0,2.0,2.0,2.0,NaN,2.0,1.0,9.0,9.0,9.0,9.0,8.0,8.0,7.0,5.0,NaN,5.0,2.0,2.0,2.0,2.0,2.0,1.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,10.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0